# Lib


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.fft import fft, fftshift, ifft, ifftshift
import plotly.express as xp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split as tts
from pylab import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle
import copy
# enc=LabelEncoder()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **CLIENT Function**

In [ ]:
# >> FEATURE SELECTION << #
def remove_correlated_features_client(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped


def remove_less_significant_features_client(X, Y):
    sl = 0.1
    regression_ols = None
    columns_dropped = np.array([])
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    regression_ols.summary()
    return columns_dropped


In [ ]:
# >> MODEL TRAINING << #
def compute_cost_client(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost


def calculate_cost_gradient_client(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array
    # print(Y_batch)
    # print(X_batch)
    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))
    # print(distance)
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw


def sgd_client(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient_client(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost_client(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights


In [ ]:
# set hyper-parameters
regularization_strength = 10000
learning_rate = 0.000001

In [ ]:
#  function for testing data ===>> computing
def prediction_generation_client(w, X):
    # calculate y_test_predicted array
    y_predicted = np.array([])
    for i in range(X.shape[0]):
        yp1_data1 = np.sign(np.dot(X.to_numpy()[i], w))
        y_predicted = np.append(y_predicted, yp1_data1)
    return y_predicted

In [ ]:
# function to return the accuray
def results_client(y_test , y_test_predicted):
    accu = accuracy_score(y_test, y_test_predicted)
    recall = recall_score(y_test, y_test_predicted)
    precision = precision_score(y_test, y_test_predicted)
    return accu , recall , precision

In [ ]:
def sign_client(w):
    size = w.size
    sign_data = np.zeros((size,))

    for i in range(0,size):
      if w[i] > 0:
        sign_data[i] = +1
      else :
        sign_data[i] = -1

    return sign_data

In [ ]:
def model_client(data):
  # step 1 replace
  data.replace({'diagnosis':{'M':1.0,'B':-1.0}},inplace=True)
  # step 2 drop
  data.drop('id',axis=1,inplace=True)
  data.drop('Unnamed: 32',axis=1,inplace=True)
  # step 3 collecting column headers
  column_headers = list(data.columns.values)
  # step 4 generating X and Y
  Y = data.loc[:, 'diagnosis']
  X = data.iloc[:, 1:]
  # step 5 filter features
  remove_correlated_features_client(X)
  remove_less_significant_features_client(X, Y)
  # step 6 normalize data for better convergence and to prevent overflow
  X_normalized = MinMaxScaler().fit_transform(X.values)
  X = pd.DataFrame(X_normalized)
  X.insert(loc=len(X.columns), column='intercept', value=1)
  # step 7 split data into train and test set
  print("splitting dataset into train and test sets...")
  X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)
  # step 8 training model
  print("training started...")
  w = sgd_client(X_train.to_numpy(), y_train.to_numpy())
  print("training finished.")
  print("weights are: {}".format(w))
  print(f"Size of w : {w.size}")
  # step 9 testing the model
  print("testing the model...")
  y_train_predicted = prediction_generation_client(w,X_train)
  y_test_predicted = prediction_generation_client(w,X_test)
  # step 10 generating results
  acc , recall , precision = results_client(y_test, y_test_predicted)
  print(f"accuracy on test dataset: {acc}")
  # step 11 geneerating sign array
  sign_data = sign_client(w)
  # step 12 spliting the weights array for compressing sensing
  w = np.array_split(w, 2)
  w_part_1 = w[0]
  w_part_2 = w[1]
  print(w_part_1)
  print(w_part_2)
  return y_train_predicted , y_test_predicted , w , w_part_1 , w_part_2 , acc , sign_data

In [ ]:
def fftc_client(x):
    """Computes the centered Fourier transform"""
    return fftshift( fft(x) )

def ifftc_client(X):
    """Inverses the centered Fourier transform"""
    return ifft( ifftshift(X) )

In [ ]:
def modification_client(x):
  x = x * pow(10,-1);
  return x

In [ ]:
def CompressSensing_1_client(w1):
  l = (w1.size + 1) * 10  # 80
  sigma = 0.03
  np.random.seed(42)
  #generate sparse signal
  x = abs(w1)
  temp = []
  const = 0
  for i in range(1,(w1.size * 10) + 1):     # 1 to 71
    if i%10 == 0:
      temp.append(x[const])
      const = const + 1
    else :
      temp.append(0)
  for i in range(0,10):
    temp.append(0)
  x = np.array(temp)
  x = modification_client(x);
  # x = x * pow(10,-1)    # modification ==>> 1

  # add random noise
  y = x + sigma * np.random.randn(l)

  fig = go.Figure()
  fig.add_trace(
      go.Scatter( x=  np.arange(l) , y = x , name='x')
  )
  fig.add_trace(
      go.Scatter( x=  np.arange(l) , y = y , name='y')
  )
  fig.show()

  # checking value of lambda
  fig = go.Figure()
  for lam in [0.01,0.05,0.03,0.025,0.035]:
      fig.add_trace(
          go.Scatter( x=  np.arange(l) , y = 1/(1+lam) * y , name=f"lambda = {str(lam)}")
      )
  fig.show()
  # taking Fourier transform
  X = fftc_client(x)
  # Y = fftc(y)
  fig = go.Figure()
  fig.add_trace( go.Scatter(x = np.arange(-l/2,l-1),y=abs(X) , name='X') )
  # fig.add_trace( go.Scatter(x = np.arange(-l/2,l-1),y=abs(Y) , name='Y') )
  fig.show()
  #uniformly sampled k-space
  Xu = 4 * X
  for i in range(1,4):
      Xu[i::4] = 0
  #reconstructed signal
  xu = ifftc_client(Xu)

  #randomly sampled k-space
  Xr = 4 * X * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l/10) )
  xr = ifftc_client( Xr )
  # X
  # plot the comparison
  fig = go.Figure()
  fig.add_trace( go.Scatter(y=x*1.5, name='original signal (scaled)'))
  fig.add_trace( go.Scatter(y=xu.real, name='uniform sampling'))
  fig.add_trace( go.Scatter(y=xr.real, name='random sampling'))
  fig.show()
  temp = 4 * X * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l/10) )
  print("temp")
  print(temp)
  print(f'Size of weights array to send after compress sensing : {temp.nbytes}')
  return temp , x

In [ ]:
def CompressSensing_2_client(w2):
  l2 = (w2.size + 1) * 10     # 70
  sigma = 0.03
  np.random.seed(42)
  #generate sparse signal
  x2 = abs(w2)
  temp2 = []
  const2 = 0
  for i in range(1,(w2.size * 10) + 1):
    if i%10 == 0:
      temp2.append(x2[const2])
      const2 = const2 + 1
    else :
      temp2.append(0)
  for i in range(0,10):
    temp2.append(0)
  x2 = np.array(temp2)

  x2  = modification_client(x2)      # x2= x2 * pow(10,-1)   modification 1

  # add random noise
  y2 = x2 + sigma * np.random.randn(l2)

  fig = go.Figure()
  fig.add_trace(
      go.Scatter( x=  np.arange(l2) , y = x2 , name='x')
  )
  fig.add_trace(
      go.Scatter( x=  np.arange(l2) , y = y2 , name='y')
  )
  fig.show()

  fig = go.Figure()
  for lam in [0.01,0.05,0.03,0.025,0.035]:
      fig.add_trace(
          go.Scatter( x=  np.arange(l2) , y = 1/(1+lam) * y2 , name=f"lambda = {str(lam)}")
      )
  fig.show()
  # taking Fourier transform
  X2 = fftc_client(x2)
  # Y = fftc(y)
  fig = go.Figure()
  fig.add_trace( go.Scatter(x = np.arange(-l2/2,l2-1),y=abs(X2) , name='X') )
  # fig.add_trace( go.Scatter(x = np.arange(-l/2,l-1),y=abs(Y) , name='Y') )
  fig.show()
  #uniformly sampled k-space
  Xu2 = 4 * X2
  for i in range(1,4):
      Xu2[i::4] = 0
  #reconstructed signal
  xu2 = ifftc_client(Xu2)

  #randomly sampled k-space
  Xr2 = 4 * X2 * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  xr2 = ifftc_client( Xr2 )
  # X
  # plot the comparison
  fig = go.Figure()
  fig.add_trace( go.Scatter(y=x2*1.5, name='original signal (scaled)'))
  fig.add_trace( go.Scatter(y=xu2.real, name='uniform sampling'))
  fig.add_trace( go.Scatter(y=xr2.real, name='random sampling'))
  fig.show()
  temp2 = 4 * X2 * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  print("temp")
  print(temp2)
  print(f'Size of weights array to send after compress sensing : {temp2.nbytes}')
  return temp2 , x2

In [ ]:
def fftc(x):
    """Computes the centered Fourier transform"""
    return fftshift( fft(x) )

def ifftc(X):
    """Inverses the centered Fourier transform"""
    return ifft( ifftshift(X) )


In [ ]:
def soft_thresh_client(x, lam):
    if ~(isinstance(x[0], complex)):
        return np.zeros(x.shape) + (x + lam) * (x<-lam) + (x - lam) * (x>lam)
    else:
        return np.zeros(x.shape) + ( abs(x) - lam ) / abs(x) * x * (abs(x)>lam)

In [ ]:
def error_graph_client(err):
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = err) )
  fig.update_layout( title = 'Error at each step' )
  fig.show()

In [ ]:
def L1_norm1_client(temp , x):
  Y = temp             # Xr
  Xhat = Y.copy()

  # Repeat steps 1-4 until change is below a threshold
  eps = 1e-4
  lam = 0.03

  def distance(x,y):
      return abs(sum(x-y))
  diff=[]
  # for err graph we have to send x array i.e ==>>
  err = []
  itermax = 10000
  while True:
      itermax -= 1
      xhat_old = ifftc_client(Xhat)
      xhat = soft_thresh_client(xhat_old, lam)
      diff.append(distance(xhat, xhat_old))
      err.append(distance(xhat.real/4,x))
      if ( diff[-1] < eps ) | ( itermax == 0 ):
          break
      Xhat = fftc_client(xhat)
      Xhat[Y!=0] = Y[Y!=0]


  #comparision graph
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = x , name = 'true signal') )
  fig.add_trace( go.Scatter( y = ifftc_client(Y).real, name = 'reconstruction before noise reduction'))
  fig.add_trace( go.Scatter( y = xhat.real/4, name = 'reconstructed after noise reduction'))
  fig.show()

  # error graph
  # error_graph_client(err)
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = err) )
  fig.update_layout( title = 'Error at each step' )
  fig.show()

  temp_CS = np.copy(xhat.real/4)
  # temp_CS contains of the reconsisted array without and modification we have done on the array for compress sensing
  return(temp_CS)

In [ ]:
def L1_norm2_client(temp , x):
  # Y = temp             # Xr temp2 = 4 * X2 * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  # Xhat = Y.copy()

  # # Repeat steps 1-4 until change is below a threshold
  # eps = 1e-4
  # lam = 0.03

  # def distance(x,y):
  #     return abs(sum(x-y))
  # diff=[]
  # # for err graph we have to send x array i.e ==>>
  # err = []
  # itermax = 10000
  # while True:
  #     itermax -= 1
  #     xhat_old = ifftc(Xhat)
  #     xhat = soft_thresh(xhat_old, lam)
  #     diff.append(distance(xhat, xhat_old))
  #     err.append(distance(xhat.real/4,x))
  #     if ( diff[-1] < eps ) | ( itermax == 0 ):
  #         break
  #     Xhat = fftc(xhat)
  #     Xhat[Y!=0] = Y[Y!=0]


  l2 = x.size
  # undersampled noisy signal in k-space and let this be first order Xhat
  # Y2 = 4 * fftc(x) * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  Y2 = temp
  Xhat2 = Y2.copy()


  # Repeat steps 1-4 until change is below a threshold
  eps = 1e-4
  lam = 0.03

  def distance(x,y):
      return abs(sum(x-y))
  diff2=[]
  err2 = []
  itermax2 = 10000
  while True:
      itermax2 -= 1
      xhat_old2 = ifftc_client(Xhat2)
      xhat2 = soft_thresh_client(xhat_old2, lam)
      diff2.append(distance(xhat2, xhat_old2))
      err2.append(distance(xhat2.real/4,x))
      if ( diff2[-1] < eps ) | ( itermax2 == 0 ):
          break
      Xhat2 = fftc_client(xhat2)
      Xhat2[Y2!=0] = Y2[Y2!=0]


  #comparision graph
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = x , name = 'true signal') )
  fig.add_trace( go.Scatter( y = ifftc_client(Y2).real, name = 'reconstruction before noise reduction'))
  fig.add_trace( go.Scatter( y = xhat2.real/4, name = 'reconstructed after noise reduction'))
  fig.show()

  # error graph
  # error_graph_client(err)
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = err2) )
  fig.update_layout( title = 'Error at each step' )
  fig.show()

  temp_CS = np.copy(xhat2.real/4)
  # temp_CS contains of the reconsisted array without and modification we have done on the array for compress sensing
  return(temp_CS)

In [ ]:
# function to regenrate weights
def construct_1_client(temp_CS):
  original_weights_size = int((temp_CS.size / 10) - 1)
  temp_w = []
  for i in temp_CS:
    if i != 0:
      temp_w.append(i)
  temp_w_CS = []
  for i in range(0,original_weights_size * 2):
    if i%2 != 0:
      temp_w_CS.append(temp_w[i])
  print(f'Weights 1 : {temp_w_CS}')
  return temp_w_CS

In [ ]:
def construct_2_client(temp_CS):
  original_weights_size = int((temp_CS.size / 10) - 1)
  temp_w2 = []
  for i in temp_CS:
    if i != 0:
      temp_w2.append(i)
  temp_w2_CS = []
  for i in range(0,original_weights_size * 2):
    if i%2 != 0:
      temp_w2_CS.append(temp_w2[i])
  print(f'Weights 1 : {temp_w2_CS}')
  return temp_w2_CS

In [ ]:
def concentrate_client(temp_w_CS, temp_w2):
  w_CS = np.concatenate((temp_w_CS, temp_w2))
  print(f'Concentrated weights : {w_CS}')
  return w_CS

In [ ]:
def reverse_modification_client(w_CS , sign):
  for i in range(0,w_CS.size):
    if sign[i] > 0:
      w_CS[i] = (+1)*(w_CS[i])
    else :
      w_CS[i] = (-1)*(w_CS[i])
  half_range = int(w_CS.size / 2)
  for i in range(0,half_range + 1):
    w_CS[i] = w_CS[i] * 4
  for i in range(half_range+1 , w_CS.size):
    w_CS[i] = w_CS[i] * 4
  # increase the weights by gloabal scaldown avg = 56.06
  for i in range(0,w_CS.size):
    w_CS[i] = w_CS[i] + ((w_CS[i] * 56.06)/100)
  print(f'reverse modified weights : {w_CS}')
  return w_CS

In [ ]:
#  function for testing data ===>> computing
def prediction_generation_client(w, X):
    # calculate y_test_predicted array
    y_predicted = np.array([])
    for i in range(X.shape[0]):
        yp1_data1 = np.sign(np.dot(X.to_numpy()[i], w))
        y_predicted = np.append(y_predicted, yp1_data1)
    return y_predicted

In [ ]:
# function to return the accuray
def results_client(y_test , y_test_predicted):
    accu = accuracy_score(y_test, y_test_predicted)
    recall = recall_score(y_test, y_test_predicted)
    precision = precision_score(y_test, y_test_predicted)
    return accu , recall , precision

In [ ]:
def partial_model_client(data,w):
  # # step 1 replace
  # data.replace({'diagnosis':{'M':1.0,'B':-1.0}},inplace=True)
  # # step 2 drop
  # data.drop('id',axis=1,inplace=True)
  # data.drop('Unnamed: 32',axis=1,inplace=True)
  # # step 3 collecting column headers
  column_headers = list(data.columns.values)
  # step 4 generating X and Y
  Y = data.loc[:, 'diagnosis']
  X = data.iloc[:, 1:]
  # step 5 filter features
  remove_correlated_features_client(X)
  remove_less_significant_features_client(X, Y)
  # step 6 normalize data for better convergence and to prevent overflow
  X_normalized = MinMaxScaler().fit_transform(X.values)
  X = pd.DataFrame(X_normalized)
  X.insert(loc=len(X.columns), column='intercept', value=1)
  # step 7 split data into train and test set
  print("splitting dataset into train and test sets...")
  X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)
  # step 8 testing the model
  print("testing the model...")
  y_train_predicted = prediction_generation_client(w,X_train)
  y_test_predicted = prediction_generation_client(w,X_test)
  # step 10 generating results
  acc , recall , precision = results_client(y_test, y_test_predicted)
  print(f"accuracy on test dataset \n accuracy: {acc} \n recall : {recall} \n precision: {precision}")

# **SERVER Functions**

In [ ]:
def fftc_server(x):
    """Computes the centered Fourier transform"""
    return fftshift( fft(x) )

def ifftc_server(X):
    """Inverses the centered Fourier transform"""
    return ifft( ifftshift(X) )


In [ ]:
def soft_thresh_server(x, lam):
    if ~(isinstance(x[0], complex)):
        return np.zeros(x.shape) + (x + lam) * (x<-lam) + (x - lam) * (x>lam)
    else:
        return np.zeros(x.shape) + ( abs(x) - lam ) / abs(x) * x * (abs(x)>lam)

In [ ]:
def error_graph_server(err):
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = err) )
  fig.update_layout( title = 'Error at each step' )
  fig.show()

In [ ]:
def L1_norm1_server(temp , x):
  # l1 = x.size
  # Y = 4 * fftc_server(x) * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l1/10) )
  Y = temp             # Xr
  Xhat = Y.copy()

  # Repeat steps 1-4 until change is below a threshold
  eps = 1e-4
  lam = 0.03

  def distance_server_norm1(x,y):
      return abs(sum(x-y))
  diff=[]
  # for err graph we have to send x array i.e ==>>
  err = []
  itermax = 10000
  while True:
      itermax -= 1
      xhat_old = ifftc_server(Xhat)
      xhat = soft_thresh_server(xhat_old, lam)
      diff.append(distance_server_norm1(xhat, xhat_old))
      err.append(distance_server_norm1(xhat.real/4,x))
      if ( diff[-1] < eps ) | ( itermax == 0 ):
          break
      Xhat = fftc_server(xhat)
      Xhat[Y!=0] = Y[Y!=0]


  #comparision graph
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = x , name = 'true signal') )
  fig.add_trace( go.Scatter( y = ifftc_server(Y).real, name = 'reconstruction before noise reduction'))
  fig.add_trace( go.Scatter( y = xhat.real/4, name = 'reconstructed after noise reduction'))
  fig.show()

  # error graph
  # error_graph(err)
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = err) )
  fig.update_layout( title = 'Error at each step' )
  fig.show()

  temp_CS = np.copy(xhat.real/4)
  # temp_CS contains of the reconsisted array without and modification we have done on the array for compress sensing
  return(temp_CS)

In [ ]:
def L1_norm2_server(temp , x):
  # Y = temp             # Xr temp2 = 4 * X2 * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  # Xhat = Y.copy()

  # # Repeat steps 1-4 until change is below a threshold
  # eps = 1e-4
  # lam = 0.03

  # def distance(x,y):
  #     return abs(sum(x-y))
  # diff=[]
  # # for err graph we have to send x array i.e ==>>
  # err = []
  # itermax = 10000
  # while True:
  #     itermax -= 1
  #     xhat_old = ifftc(Xhat)
  #     xhat = soft_thresh_server(xhat_old, lam)
  #     diff.append(distance(xhat, xhat_old))
  #     err.append(distance(xhat.real/4,x))
  #     if ( diff[-1] < eps ) | ( itermax == 0 ):
  #         break
  #     Xhat = fftc(xhat)
  #     Xhat[Y!=0] = Y[Y!=0]


  # l2 = x.size
  # undersampled noisy signal in k-space and let this be first order Xhat
  # Y2 = 4 * fftc_server(x) * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  Y2 = temp
  Xhat2 = Y2.copy()


  # Repeat steps 1-4 until change is below a threshold
  eps = 1e-4
  lam = 0.03

  def distance_server_norm2(x,y):
      return abs(sum(x-y))
  diff2=[]
  err2 = []
  itermax2 = 10000
  while True:
      itermax2 -= 1
      xhat_old2 = ifftc_server(Xhat2)
      xhat2 = soft_thresh_server(xhat_old2, lam)
      diff2.append(distance_server_norm2(xhat2, xhat_old2))
      err2.append(distance_server_norm2(xhat2.real/4,x))
      if ( diff2[-1] < eps ) | ( itermax2 == 0 ):
          break
      Xhat2 = fftc_server(xhat2)
      Xhat2[Y2!=0] = Y2[Y2!=0]


  #comparision graph
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = x , name = 'true signal') )
  fig.add_trace( go.Scatter( y = ifftc(Y2).real, name = 'reconstruction before noise reduction'))
  fig.add_trace( go.Scatter( y = xhat2.real/4, name = 'reconstructed after noise reduction'))
  fig.show()

  # error graph
  # error_graph(err)
  fig = go.Figure()
  fig.add_trace( go.Scatter( y = err2) )
  fig.update_layout( title = 'Error at each step' )
  fig.show()

  temp_CS = np.copy(xhat2.real/4)
  # temp_CS contains of the reconsisted array without and modification we have done on the array for compress sensing
  return(temp_CS)

In [ ]:
# function to regenrate weights
def construct_1_server(temp_CS):
  original_weights_size = int((temp_CS.size / 10) - 1)
  temp_w = []
  for i in temp_CS:
    if i != 0:
      temp_w.append(i)
  temp_w_CS = []
  for i in range(0,original_weights_size * 2):
    if i%2 != 0:
      temp_w_CS.append(temp_w[i])
  print(f'Weights 1 : {temp_w_CS}')
  return temp_w_CS

In [ ]:
def construct_2_server(temp_CS):
  original_weights_size = int((temp_CS.size / 10) - 1)
  temp_w2 = []
  for i in temp_CS:
    if i != 0:
      temp_w2.append(i)
  temp_w2_CS = []
  for i in range(0,original_weights_size * 2):
    if i%2 != 0:
      temp_w2_CS.append(temp_w2[i])
  print(f'Weights 1 : {temp_w2_CS}')
  return temp_w2_CS

In [ ]:
def concentrate_server(temp_w_CS, temp_w2):
  w_CS = np.concatenate((temp_w_CS, temp_w2))
  print(f'Combined Weights : {w_CS}')
  return w_CS

In [ ]:
def reverse_modification_server(w_CS , sign):
  for i in range(0,w_CS.size):
    if sign[i] > 0:
      w_CS[i] = (+1)*(w_CS[i])
    else :
      w_CS[i] = (-1)*(w_CS[i])
  w_CS = w_CS * pow(10,1)
  # we have to increase by a global scalled down weightage = 56.06
  for i in range(0,w_CS.size):
    w_CS[i] = w_CS[i] + ((w_CS[i] * 56.06)/100)
  print(f'Reverse Modified : {w_CS}')
  return w_CS

In [ ]:
def aggregation_server5(w_client1_CS , w_client2_CS , w_client3_CS , w_client4_CS , w_client5_CS):
  print('Global weights ---->>> average taken')
  size = w_client1_CS.size
  weights_aggregated = np.zeros(size)
  print('Aggregating ==>> taking averge')
  for i in range(0,size):
    weights_aggregated[i] = (w_client1_CS[i] + w_client2_CS[i] + w_client3_CS[i] + w_client4_CS[i] + w_client5_CS[i])/5
  return weights_aggregated

In [ ]:
def aggregation_server4(w_client1_CS , w_client2_CS , w_client3_CS , w_client4_CS , w_client5_CS):
  print('Global weights ---->>> average taken')
  size = w_client1_CS.size
  weights_aggregated = np.zeros(size)
  print('Aggregating ==>> taking averge')
  for i in range(0,size):
    weights_aggregated[i] = (w_client1_CS[i] + w_client2_CS[i] + w_client3_CS[i] + w_client4_CS[i] )/4
  return weights_aggregated

In [ ]:
def aggregation_server3(w_client1_CS , w_client2_CS , w_client3_CS , w_client4_CS , w_client5_CS):
  print('Global weights ---->>> average taken')
  size = w_client1_CS.size
  weights_aggregated = np.zeros(size)
  print('Aggregating ==>> taking averge')
  for i in range(0,size):
    weights_aggregated[i] = (w_client1_CS[i] + w_client2_CS[i] + w_client3_CS[i])/3
  return weights_aggregated

In [ ]:
def aggregation_server2(w_client1_CS , w_client2_CS , w_client3_CS , w_client4_CS , w_client5_CS):
  print('Global weights ---->>> average taken')
  size = w_client1_CS.size
  weights_aggregated = np.zeros(size)
  print('Aggregating ==>> taking averge')
  for i in range(0,size):
    weights_aggregated[i] = (w_client1_CS[i] + w_client2_CS[i] )/2
  return weights_aggregated

In [ ]:
def sign_server(w):
    size = w.size
    sign_data = np.zeros((size,))

    for i in range(0,size):
      if w[i] > 0:
        sign_data[i] = +1
      else :
        sign_data[i] = -1

    return sign_data

In [ ]:
def split_server(w):
  w = np.array_split(w, 2)
  w_part_1 = w[0]
  w_part_2 = w[1]
  return w_part_1 , w_part_2

In [ ]:
def fftc(x):
    """Computes the centered Fourier transform"""
    return fftshift( fft(x) )

def ifftc(X):
    """Inverses the centered Fourier transform"""
    return ifft( ifftshift(X) )

In [ ]:
# have to figure out this fucntion based on the
def modification_server(x):
  x = x / 4;
  return x

In [ ]:
# have to figure out this fucntion based on the
def modification2_server(x):
  x = x / 4;
  return x

In [ ]:
def CompressSensing_1_server(w1):
  l = (w1.size + 1) * 10  # 80
  sigma = 0.03
  np.random.seed(42)
  #generate sparse signal
  x = abs(w1)
  temp = []
  const = 0
  for i in range(1,(w1.size * 10) + 1):     # 1 to 71
    if i%10 == 0:
      temp.append(x[const])
      const = const + 1
    else :
      temp.append(0)
  for i in range(0,10):
    temp.append(0)
  x = np.array(temp)
  x = modification_server(x);
  # x = x * pow(10,-1)    # modification ==>> 1

  # add random noise
  y = x + sigma * np.random.randn(l)

  fig = go.Figure()
  fig.add_trace(
      go.Scatter( x=  np.arange(l) , y = x , name='x')
  )
  fig.add_trace(
      go.Scatter( x=  np.arange(l) , y = y , name='y')
  )
  fig.show()

  # checking value of lambda
  fig = go.Figure()
  for lam in [0.01,0.05,0.03,0.025,0.035]:
      fig.add_trace(
          go.Scatter( x=  np.arange(l) , y = 1/(1+lam) * y , name=f"lambda = {str(lam)}")
      )
  fig.show()
  # taking Fourier transform
  X = fftc_server(x)
  # Y = fftc(y)
  fig = go.Figure()
  fig.add_trace( go.Scatter(x = np.arange(-l/2,l-1),y=abs(X) , name='X') )
  # fig.add_trace( go.Scatter(x = np.arange(-l/2,l-1),y=abs(Y) , name='Y') )
  fig.show()
  #uniformly sampled k-space
  Xu = 4 * X
  for i in range(1,4):
      Xu[i::4] = 0
  #reconstructed signal
  xu = ifftc_server(Xu)

  #randomly sampled k-space
  Xr = 4 * X * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l/10) )
  xr = ifftc_server( Xr )
  # X
  # plot the comparison
  fig = go.Figure()
  fig.add_trace( go.Scatter(y=x*1.5, name='original signal (scaled)'))
  fig.add_trace( go.Scatter(y=xu.real, name='uniform sampling'))
  fig.add_trace( go.Scatter(y=xr.real, name='random sampling'))
  fig.show()
  temp = 4 * X * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l/10) )
  print("temp")
  print(temp)
  print(f'Size of weights array to send after compress sensing : {temp.nbytes}')
  return temp , x

In [ ]:
def CompressSensing_2_server(w2):
  l2 = (w2.size + 1) * 10     # 70
  sigma = 0.03
  np.random.seed(42)
  #generate sparse signal
  x2 = abs(w2)
  temp2 = []
  const2 = 0
  for i in range(1,(w2.size * 10) + 1):
    if i%10 == 0:
      temp2.append(x2[const2])
      const2 = const2 + 1
    else :
      temp2.append(0)
  for i in range(0,10):
    temp2.append(0)
  x2 = np.array(temp2)

  x2  = modification2_server(x2)      # x2= x2 * pow(10,-1)   modification 1

  # add random noise
  y2 = x2 + sigma * np.random.randn(l2)

  fig = go.Figure()
  fig.add_trace(
      go.Scatter( x=  np.arange(l2) , y = x2 , name='x')
  )
  fig.add_trace(
      go.Scatter( x=  np.arange(l2) , y = y2 , name='y')
  )
  fig.show()

  fig = go.Figure()
  for lam in [0.01,0.05,0.03,0.025,0.035]:
      fig.add_trace(
          go.Scatter( x=  np.arange(l2) , y = 1/(1+lam) * y2 , name=f"lambda = {str(lam)}")
      )
  fig.show()
  # taking Fourier transform
  X2 = fftc_server(x2)
  # Y = fftc(y)
  fig = go.Figure()
  fig.add_trace( go.Scatter(x = np.arange(-l2/2,l2-1),y=abs(X2) , name='X') )
  # fig.add_trace( go.Scatter(x = np.arange(-l/2,l-1),y=abs(Y) , name='Y') )
  fig.show()
  #uniformly sampled k-space
  Xu2 = 4 * X2
  for i in range(1,4):
      Xu2[i::4] = 0
  #reconstructed signal
  xu2 = ifftc_server(Xu2)

  #randomly sampled k-space
  Xr2 = 4 * X2 * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  xr2 = ifftc_server( Xr2 )
  # X
  # plot the comparison
  fig = go.Figure()
  fig.add_trace( go.Scatter(y=x2*1.5, name='original signal (scaled)'))
  fig.add_trace( go.Scatter(y=xu2.real, name='uniform sampling'))
  fig.add_trace( go.Scatter(y=xr2.real, name='random sampling'))
  fig.show()
  temp2 = 4 * X2 * np.random.permutation(np.repeat([0,0,0,0,0,0,0,0,0,1], l2/10) )
  print("temp")
  print(temp2)
  print(f'Size of weights array to send after compress sensing : {temp2.nbytes}')
  return temp2 , x2

# CREATE DATA

In [ ]:
data_main = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Breast_Cancer.csv')
# data =pd.read_csv('healthcare-dataset-stroke-data.csv.xls')

In [ ]:
Book1 = data_main

In [ ]:
Book2 = data_main.sample(frac = 0.98 , random_state = 250)

In [ ]:
Book3 = data_main.sample(frac = 0.96 , random_state = 350)

In [ ]:
Book4 = data_main

In [ ]:
Book5 = data_main.sample(frac = 0.95 , random_state = 350)

In [ ]:
Book1.reset_index()
Book2.reset_index()
Book3.reset_index()
Book4.reset_index()
Book5.reset_index()
Book1.to_excel('/content/drive/MyDrive/Colab Notebooks/Book1.xlsx', index=False)
Book2.to_excel('/content/drive/MyDrive/Colab Notebooks/Book2.xlsx', index=False)
Book3.to_excel('/content/drive/MyDrive/Colab Notebooks/Book3.xlsx', index=False)
Book4.to_excel('/content/drive/MyDrive/Colab Notebooks/Book4.xlsx', index=False)
Book5.to_excel('/content/drive/MyDrive/Colab Notebooks/Book5.xlsx', index=False)

# Reading Data

In [ ]:
data1 =pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Book1.xlsx')
data2 =pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Book2.xlsx')
data3 =pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Book3.xlsx')
data4 =pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Book4.xlsx')
data5 =pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Book5.xlsx')

In [ ]:
print(data1.shape)
print(data2.shape)
print(data3.shape)
print(data4.shape)
print(data5.shape)

(569, 33)
(558, 33)
(546, 33)
(569, 33)
(541, 33)


# Client-1 Calling

In [ ]:
# calling model for client
y_train_predicted_client1 , y_test_predicted_client1 , w_client1 , w_client1_part_1 , w_client1_part_2 , acc_client1 , sign_data_client1 = model_client(data1)

splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 7065.221921360915
Epoch is: 2 and Cost is: 6242.657941355712
Epoch is: 4 and Cost is: 4863.855975704807
Epoch is: 8 and Cost is: 3402.5378148081286
Epoch is: 16 and Cost is: 2485.4940179109394
Epoch is: 32 and Cost is: 1930.6516182597422
Epoch is: 64 and Cost is: 1562.522916153761
Epoch is: 128 and Cost is: 1322.8464265165164
Epoch is: 256 and Cost is: 1147.4349304679536
Epoch is: 512 and Cost is: 1055.526944418695
Epoch is: 1024 and Cost is: 1019.4467756932434
Epoch is: 2048 and Cost is: 1018.7234317865747
training finished.
weights are: [ 3.90570889  9.45967278 -1.94654251 -6.87880976 10.45060558 -2.08683319
 -7.5908927   2.54983458 -3.05369654  2.73635443  3.34831441  5.26789483
  2.46712325 -5.0558692 ]
Size of w : 14
testing the model...
accuracy on test dataset: 0.9736842105263158
[ 3.90570889  9.45967278 -1.94654251 -6.87880976 10.45060558 -2.08683319
 -7.5908927 ]
[ 2.54983458 -3.05369654

In [ ]:
# calling compress sensing for client
client1_temp1 , client1_x1 = CompressSensing_1_client( w_client1_part_1 )
client1_temp2 , client1_x2 = CompressSensing_2_client( w_client1_part_2 )

temp
[  0.        -0.j           0.        +0.j
   0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j           0.        +0.j
 -13.69473724-9.94980901j   0.        +0.j
  -0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j           0.        +0.j
   0.        -0.j           0.        +0.j
  -0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
  -0.        +0.j           0.        +0.j
  -0.        +0.j          -6.4856952 -2.25578346j
   0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        -0.j           0.14118752+6.86533811j
  -0.        +0.j           0.        -0.j
  -0.        +0.j           0.        -0.j
   0.        +0.j          -0.        +0.j
   0.        -0.j          -0.        +0.j
   0.        -0.j           0.   

temp
[-0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j         -7.92159903-5.75537859j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -0.        +0.j          0.        +0.j
 -0.        +0.j          0.        -0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.        +0.j
  0.        +0.j          0.        +0.j         -0.        +0.j
  0.        +0.j         -0.        +0.j         -2.05848444-0.14502599j
 -0.        +0.j         -0.        +0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.49817877+2.00255054j
 -0.        +0.j          0.        -0.j         -0.        +0.j
  0.        -0.j          0.        +0.j         -0.        +0.j
  0.        -0.j         -0.        +0.j          0.        -

In [ ]:
# calling L1 norm
recons_client1_temp1 = L1_norm1_server(client1_temp1 , client1_x1)
recons_client1_temp2 = L1_norm2_server(client1_temp2 , client1_x2)
# print(recons_client1_temp1.size)
# print(recons_client1_temp2.size)

In [ ]:
# construct , concentrate and rever_modification call to get weights

# step 1 constructing weights
recons_cons_client1_temp1 = construct_1_server(recons_client1_temp1)
recons_cons_client1_temp2 = construct_2_server(recons_client1_temp2)

# step 2 concentrate
client1_server_concentrate = concentrate_server(recons_cons_client1_temp1, recons_cons_client1_temp2)

# step 3  reverse_modification
client1_reverse_modif = reverse_modification_server(client1_server_concentrate , sign_data_client1)

print(f'CLient 1')
client1_server_weights = copy.deepcopy(client1_reverse_modif)

Weights 1 : [0.11846039203061892, 0.4784324677703831, 0.0696391338577233, 0.3331382792835316, 0.40437631773982524, 0.1026412077923114, 0.3958485747234769]
Weights 1 : [0.09575722896180944, 0.18105324300316533, 0.07313746578217312, 0.18168311992848035, 0.17313487099804636, 0.1144633293149272, 0.19494363825833408]
Combined Weights : [0.11846039 0.47843247 0.06963913 0.33313828 0.40437632 0.10264121
 0.39584857 0.09575723 0.18105324 0.07313747 0.18168312 0.17313487
 0.11446333 0.19494364]
Reverse Modified : [ 1.84869288  7.46641709 -1.08678832 -5.19895599  6.31069681 -1.60181869
 -6.17761286  1.49438732 -2.82551691  1.14138329  2.83534677  2.7019428
  1.78631472 -3.04229042]
CLient 1


# Client-2 Calling

In [ ]:
y_train_predicted_client2 , y_test_predicted_client2 , w_client2 , w_client2_part_1 , w_client2_part_2 , acc_client2 , sign_data_client2 = model_client(data2)

splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 7165.749050348703
Epoch is: 2 and Cost is: 6347.229414907514
Epoch is: 4 and Cost is: 4790.453002566987
Epoch is: 8 and Cost is: 3403.804342873047
Epoch is: 16 and Cost is: 2571.8992300431064
Epoch is: 32 and Cost is: 1969.4577833241285
Epoch is: 64 and Cost is: 1582.06391054967
Epoch is: 128 and Cost is: 1317.1145767554492
Epoch is: 256 and Cost is: 1148.2011538667211
Epoch is: 512 and Cost is: 1056.1355067292684
Epoch is: 1024 and Cost is: 1004.6046694601747
Epoch is: 2048 and Cost is: 995.3981553260737
training finished.
weights are: [ 3.51935374 10.44833288 -2.14710071 -8.8658669   9.61737835 -1.3948735
 -8.3305339   2.76683257 -1.41822112  3.60433228  1.7621213   4.9546517
  3.47408705 -4.90313848]
Size of w : 14
testing the model...
accuracy on test dataset: 0.9553571428571429
[ 3.51935374 10.44833288 -2.14710071 -8.8658669   9.61737835 -1.3948735
 -8.3305339 ]
[ 2.76683257 -1.41822112  3.6

In [ ]:
client2_temp1 , client2_x1 = CompressSensing_1_client( w_client2_part_1 )
client2_temp2 , client2_x2 = CompressSensing_2_client( w_client2_part_2 )

temp
[ -0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
 -14.34336648-10.42106574j   0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j           0.         +0.j
   0.         -0.j           0.         +0.j
  -0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
  -0.         +0.j           0.         +0.j
  -0.         +0.j          -7.71401668 -1.89286932j
  -0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j          -0.58353655 +7.92139462j
  -0.         +0.j           0.         -0.j
  -0.         +0.j          -0.         +0.j
   0.         +0.j           0.         -0.j
   0.         -0.j          -0.   

temp
[-0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -0.        +0.j         -7.40521878-5.38020637j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.        +0.j          0.        -0.j          0.        +0.j
  0.        +0.j          0.        +0.j         -0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.        +0.j
  0.        +0.j          0.        +0.j         -0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.41902809+0.23837361j
 -0.        +0.j          0.        -0.j          0.        +0.j
  0.        +0.j         -0.        +0.j         -0.35619357+0.3248579j
 -0.        +0.j          0.        -0.j          0.        -0.j
  0.        -0.j          0.        +0.j         -0.        +0.j
 -0.        +0.j         -0.        +0.j          0.        -0

In [ ]:
recons_client2_temp1 = L1_norm1_server(client2_temp1 , client2_x1)
recons_client2_temp2 = L1_norm2_server(client2_temp2 , client2_x2)

In [ ]:
# step 1 constructing weights
recons_cons_client2_temp1 = construct_1_server(recons_client2_temp1)
recons_cons_client2_temp2 = construct_2_server(recons_client2_temp2)

# step 2 concentrate
client2_server_concentrate = concentrate_server(recons_cons_client2_temp1, recons_cons_client2_temp2)

# step 3  reverse_modification
client2_reverse_modif = reverse_modification_server(client2_server_concentrate , sign_data_client2)

print(f'CLient 2')
client2_server_weights = copy.deepcopy(client2_reverse_modif)

Weights 1 : [0.11342211116065865, 0.5174718795936245, 0.08128573694262903, 0.4031038265934642, 0.38272127767960057, 0.07891618704206188, 0.45443433287381024]
Weights 1 : [0.09535355085308223, 0.1206869859187068, 0.10604717118447426, 0.13379599925907443, 0.14348196682909536, 0.1668751816692879, 0.15219876380213151]
Combined Weights : [0.11342211 0.51747188 0.08128574 0.40310383 0.38272128 0.07891619
 0.45443433 0.09535355 0.12068699 0.10604717 0.133796   0.14348197
 0.16687518 0.15219876]
Reverse Modified : [ 1.77006547  8.07566615 -1.26854521 -6.29083832  5.97274826 -1.23156601
 -7.0919022   1.48808751 -1.8834411   1.65497215  2.08802036  2.23917957
  2.60425409 -2.37521391]
CLient 2


# Client-3 Calling

In [ ]:
y_train_predicted_client3 , y_test_predicted_client3 , w_client3 , w_client3_part_1 , w_client3_part_2 , acc_client3 , sign_data_client3 = model_client(data3)

splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 7466.414931506928
Epoch is: 2 and Cost is: 6426.885319376327
Epoch is: 4 and Cost is: 4822.115659930362
Epoch is: 8 and Cost is: 3362.4297984002
Epoch is: 16 and Cost is: 2583.9116018254044
Epoch is: 32 and Cost is: 2007.6908784481816
Epoch is: 64 and Cost is: 1596.969662719394
Epoch is: 128 and Cost is: 1333.5197284987164
Epoch is: 256 and Cost is: 1173.498320756436
Epoch is: 512 and Cost is: 1071.6173078270308
Epoch is: 1024 and Cost is: 1030.7484337114427
Epoch is: 2048 and Cost is: 1018.3688903227799
Epoch is: 4096 and Cost is: 1016.562881234501
training finished.
weights are: [  3.39630492   9.60912121  -0.25466153  -5.99485239  11.07200342
  -2.19163514 -10.39433023   2.33605836  -3.85281503   1.57407136
   3.62714179   4.42184867   3.66588384  -4.9601192 ]
Size of w : 14
testing the model...
accuracy on test dataset: 0.9818181818181818
[  3.39630492   9.60912121  -0.25466153  -5.99485239  

In [ ]:
client3_temp1 , client3_x1 = CompressSensing_1_client( w_client3_part_1 )
client3_temp2 , client3_x2 = CompressSensing_2_client( w_client3_part_2 )

temp
[ -0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j           0.         +0.j
 -13.88690901-10.08942998j   0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j           0.         +0.j
  -0.         +0.j           0.         +0.j
  -0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j           0.         +0.j
  -0.         +0.j           0.         +0.j
  -0.         +0.j          -8.09771427 -2.8460952j
  -0.         +0.j           0.         -0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j           0.20446606 +8.58087571j
  -0.         +0.j           0.         -0.j
  -0.         +0.j           0.         -0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j          -0.    

temp
[-0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j         -7.90828294-5.74570388j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -0.        +0.j          0.        +0.j
 -0.        +0.j          0.        -0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.        +0.j
  0.        +0.j          0.        +0.j         -0.        +0.j
  0.        +0.j         -0.        +0.j         -2.4148171 +0.32920156j
 -0.        +0.j          0.        -0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j         -1.05930882+2.19489866j
 -0.        +0.j          0.        -0.j         -0.        +0.j
  0.        -0.j          0.        +0.j         -0.        +0.j
  0.        -0.j         -0.        +0.j          0.        -

In [ ]:
recons_client3_temp1 = L1_norm1_server(client3_temp1 , client3_x1)
recons_client3_temp2 = L1_norm2_server(client3_temp2 , client3_x2)

In [ ]:
# step 1 constructing weights
recons_cons_client3_temp1 = construct_1_server(recons_client3_temp1)
recons_cons_client3_temp2 = construct_2_server(recons_client3_temp2)

# step 2 concentrate
client3_server_concentrate = concentrate_server(recons_cons_client3_temp1, recons_cons_client3_temp2)

# step 3  reverse_modification
client3_reverse_modif = reverse_modification_server(client3_server_concentrate , sign_data_client3)

print(f'CLient 3')
client3_server_weights = copy.deepcopy(client3_reverse_modif)

Weights 1 : [0.05915251807223532, 0.528477639377714, 0.019175834257401256, 0.3089135403735486, 0.42596551836474883, 0.10398015692287596, 0.5020781430172476]
Weights 1 : [0.11152916480024971, 0.18589965495832098, 0.05092921585840258, 0.18267967758608109, 0.16001454253586372, 0.1315094151572889, 0.21315439218029847]
Combined Weights : [0.05915252 0.52847764 0.01917583 0.30891354 0.42596552 0.10398016
 0.50207814 0.11152916 0.18589965 0.05092922 0.18267968 0.16001454
 0.13150942 0.21315439]
Reverse Modified : [ 0.9231342   8.24742204 -0.29925807 -4.82090471  6.64761788 -1.62271433
 -7.8354315   1.74052415 -2.90115002  0.79480134  2.85089905  2.49718695
  2.05233593 -3.32648744]
CLient 3


# Client-4 Calling

In [ ]:
y_train_predicted_client4 , y_test_predicted_client4 , w_client4 , w_client4_part_1 , w_client4_part_2 , acc_client4 , sign_data_client4 = model_client(data4)

splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 7065.221921360915
Epoch is: 2 and Cost is: 6242.657941355712
Epoch is: 4 and Cost is: 4863.855975704807
Epoch is: 8 and Cost is: 3402.5378148081286
Epoch is: 16 and Cost is: 2485.4940179109394
Epoch is: 32 and Cost is: 1930.6516182597422
Epoch is: 64 and Cost is: 1562.522916153761
Epoch is: 128 and Cost is: 1322.8464265165164
Epoch is: 256 and Cost is: 1147.4349304679536
Epoch is: 512 and Cost is: 1055.526944418695
Epoch is: 1024 and Cost is: 1019.4467756932434
Epoch is: 2048 and Cost is: 1018.7234317865747
training finished.
weights are: [ 3.90570889  9.45967278 -1.94654251 -6.87880976 10.45060558 -2.08683319
 -7.5908927   2.54983458 -3.05369654  2.73635443  3.34831441  5.26789483
  2.46712325 -5.0558692 ]
Size of w : 14
testing the model...
accuracy on test dataset: 0.9736842105263158
[ 3.90570889  9.45967278 -1.94654251 -6.87880976 10.45060558 -2.08683319
 -7.5908927 ]
[ 2.54983458 -3.05369654

In [ ]:
client4_temp1 , client4_x1 = CompressSensing_1_client( w_client4_part_1 )
client4_temp2 , client4_x2 = CompressSensing_2_client( w_client4_part_2 )

temp
[  0.        -0.j           0.        +0.j
   0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j           0.        +0.j
 -13.69473724-9.94980901j   0.        +0.j
  -0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j           0.        +0.j
   0.        -0.j           0.        +0.j
  -0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
  -0.        +0.j           0.        +0.j
  -0.        +0.j          -6.4856952 -2.25578346j
   0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        -0.j           0.14118752+6.86533811j
  -0.        +0.j           0.        -0.j
  -0.        +0.j           0.        -0.j
   0.        +0.j          -0.        +0.j
   0.        -0.j          -0.        +0.j
   0.        -0.j           0.   

temp
[-0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j         -7.92159903-5.75537859j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -0.        +0.j          0.        +0.j
 -0.        +0.j          0.        -0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.        +0.j
  0.        +0.j          0.        +0.j         -0.        +0.j
  0.        +0.j         -0.        +0.j         -2.05848444-0.14502599j
 -0.        +0.j         -0.        +0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.49817877+2.00255054j
 -0.        +0.j          0.        -0.j         -0.        +0.j
  0.        -0.j          0.        +0.j         -0.        +0.j
  0.        -0.j         -0.        +0.j          0.        -

In [ ]:
recons_client4_temp1 = L1_norm1_server(client4_temp1 , client4_x1)
recons_client4_temp2 = L1_norm2_server(client4_temp2 , client4_x2)

In [ ]:
# step 1 constructing weights
recons_cons_client4_temp1 = construct_1_server(recons_client4_temp1)
recons_cons_client4_temp2 = construct_2_server(recons_client4_temp2)

# step 2 concentrate
client4_server_concentrate = concentrate_server(recons_cons_client4_temp1, recons_cons_client4_temp2)

# step 3  reverse_modification
client4_reverse_modif = reverse_modification_server(client4_server_concentrate , sign_data_client4)

print(f'CLient 4')
client4_server_weights = copy.deepcopy(client4_reverse_modif)

Weights 1 : [0.11846039203061892, 0.4784324677703831, 0.0696391338577233, 0.3331382792835316, 0.40437631773982524, 0.1026412077923114, 0.3958485747234769]
Weights 1 : [0.09575722896180944, 0.18105324300316533, 0.07313746578217312, 0.18168311992848035, 0.17313487099804636, 0.1144633293149272, 0.19494363825833408]
Combined Weights : [0.11846039 0.47843247 0.06963913 0.33313828 0.40437632 0.10264121
 0.39584857 0.09575723 0.18105324 0.07313747 0.18168312 0.17313487
 0.11446333 0.19494364]
Reverse Modified : [ 1.84869288  7.46641709 -1.08678832 -5.19895599  6.31069681 -1.60181869
 -6.17761286  1.49438732 -2.82551691  1.14138329  2.83534677  2.7019428
  1.78631472 -3.04229042]
CLient 4


# Client-5 Calling

In [ ]:
y_train_predicted_client5 , y_test_predicted_client5 , w_client5 , w_client5_part_1 , w_client5_part_2 , acc_client5 , sign_data_client5 = model_client(data5)

splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 7299.520280737874
Epoch is: 2 and Cost is: 6407.789675121375
Epoch is: 4 and Cost is: 4792.511569929667
Epoch is: 8 and Cost is: 3353.2543287113176
Epoch is: 16 and Cost is: 2517.838025977372
Epoch is: 32 and Cost is: 1959.06568133116
Epoch is: 64 and Cost is: 1546.5132726492172
Epoch is: 128 and Cost is: 1274.6131242731453
Epoch is: 256 and Cost is: 1111.8657502847434
Epoch is: 512 and Cost is: 1010.7943771162537
Epoch is: 1024 and Cost is: 962.9676868879912
Epoch is: 2048 and Cost is: 942.6081050111354
Epoch is: 4096 and Cost is: 944.9438341893308
training finished.
weights are: [  3.5325488   10.12431885  -0.38787854  -6.77067625  10.79583822
  -2.81029751 -10.01354773   2.16404387  -2.6934619    1.9888862
   3.14657529   4.91317983   4.03658711  -5.18029939]
Size of w : 14
testing the model...
accuracy on test dataset: 0.963302752293578
[  3.5325488   10.12431885  -0.38787854  -6.77067625  10

In [ ]:
client5_temp1 , client5_x1 = CompressSensing_1_client( w_client5_part_1 )
client5_temp2 , client5_x2 = CompressSensing_2_client( w_client5_part_2 )

temp
[ -0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
 -14.37950233-10.44731997j   0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j           0.         +0.j
  -0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j           0.         +0.j
  -0.         +0.j           0.         +0.j
  -0.         +0.j          -8.04987821 -2.43491684j
  -0.         +0.j           0.         -0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j          -0.17180564 +8.40831981j
  -0.         +0.j           0.         -0.j
  -0.         +0.j           0.         -0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j          -0.   

temp
[-0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -0.        +0.j         -7.80637765-5.67166535j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j         -0.        +0.j          0.        +0.j
 -0.        +0.j          0.        -0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
 -0.        +0.j         -0.        +0.j         -0.        +0.j
  0.        +0.j          0.        +0.j         -0.        +0.j
  0.        +0.j         -0.        +0.j         -1.76779465+0.30987951j
 -0.        +0.j          0.        -0.j          0.        +0.j
  0.        +0.j         -0.        +0.j         -0.84099152+1.58551458j
 -0.        +0.j          0.        -0.j         -0.        +0.j
  0.        -0.j          0.        +0.j         -0.        +0.j
  0.        -0.j         -0.        +0.j          0.        -

In [ ]:
recons_client5_temp1 = L1_norm1_server(client5_temp1 , client5_x1)
recons_client5_temp2 = L1_norm2_server(client5_temp2 , client5_x2)

In [ ]:
# step 1 constructing weights
recons_cons_client5_temp1 = construct_1_server(recons_client5_temp1)
recons_cons_client5_temp2 = construct_2_server(recons_client5_temp2)

# step 2 concentrate
client5_server_concentrate = concentrate_server(recons_cons_client5_temp1, recons_cons_client5_temp2)

# step 3  reverse_modification
client5_reverse_modif = reverse_modification_server(client5_server_concentrate , sign_data_client5)

print(f'CLient 5')
client5_server_weights = copy.deepcopy(client5_reverse_modif)

Weights 1 : [0.08775018076191107, 0.5291349335874992, 0.0336929431970418, 0.3390032318157466, 0.4251260004813753, 0.11238871087034835, 0.4991857847047945]
Weights 1 : [0.09984002361398114, 0.15545175608599884, 0.06475823514225518, 0.17060588328619078, 0.16309764543802852, 0.15320415979196264, 0.20037211152429263]
Combined Weights : [0.08775018 0.52913493 0.03369294 0.33900323 0.425126   0.11238871
 0.49918578 0.09984002 0.15545176 0.06475824 0.17060588 0.16309765
 0.15320416 0.20037211]
Reverse Modified : [ 1.36942932  8.25767977 -0.52581207 -5.29048444  6.63451636 -1.75393822
 -7.79029336  1.55810341 -2.42598011  1.01061702  2.66247541  2.54530185
  2.39090412 -3.12700717]
CLient 5


# Server Calling


In [ ]:
print(f'weights recieved are : \n "client 1 ->" {client1_server_weights} \n "client 2 ->" {client2_server_weights} \
                               \n "client 3 ->" {client3_server_weights} \n "client 4 ->" {client4_server_weights} \
                               \n "client 5 ->" {client5_server_weights} \n')

weights recieved are : 
 "client 1 ->" [ 1.84869288  7.46641709 -1.08678832 -5.19895599  6.31069681 -1.60181869
 -6.17761286  1.49438732 -2.82551691  1.14138329  2.83534677  2.7019428
  1.78631472 -3.04229042] 
 "client 2 ->" [ 1.77006547  8.07566615 -1.26854521 -6.29083832  5.97274826 -1.23156601
 -7.0919022   1.48808751 -1.8834411   1.65497215  2.08802036  2.23917957
  2.60425409 -2.37521391]                                
 "client 3 ->" [ 0.9231342   8.24742204 -0.29925807 -4.82090471  6.64761788 -1.62271433
 -7.8354315   1.74052415 -2.90115002  0.79480134  2.85089905  2.49718695
  2.05233593 -3.32648744] 
 "client 4 ->" [ 1.84869288  7.46641709 -1.08678832 -5.19895599  6.31069681 -1.60181869
 -6.17761286  1.49438732 -2.82551691  1.14138329  2.83534677  2.7019428
  1.78631472 -3.04229042]                                
 "client 5 ->" [ 1.36942932  8.25767977 -0.52581207 -5.29048444  6.63451636 -1.75393822
 -7.79029336  1.55810341 -2.42598011  1.01061702  2.66247541  2.54530185
  2

In [ ]:
# taking average or aggreagation ===>> according to the advancement
server_aggregated_weights = aggregation_server5(client1_server_weights , client2_server_weights , client3_server_weights , client4_server_weights , client5_server_weights)
print(server_aggregated_weights)
# creating sign data for server
sign_data_server = sign_server(server_aggregated_weights)
# splitting weights
server_final_weights_part_1 , server_final_weights_part_2 = split_server(server_aggregated_weights)
print(f'splitted weights ===>> {server_final_weights_part_1} \n {server_final_weights_part_2}')

Global weights ---->>> average taken
Aggregating ==>> taking averge
[ 1.55200295  7.90272043 -0.8534384  -5.36002789  6.37525523 -1.56237119
 -7.01457055  1.55509794 -2.57232101  1.14863142  2.65441767  2.53711079
  2.12402471 -2.98265787]
splitted weights ===>> [ 1.55200295  7.90272043 -0.8534384  -5.36002789  6.37525523 -1.56237119
 -7.01457055] 
 [ 1.55509794 -2.57232101  1.14863142  2.65441767  2.53711079  2.12402471
 -2.98265787]


In [ ]:
# calling compress sensing for server
server_temp1 , server_x1 = CompressSensing_1_server( server_final_weights_part_1 )
server_temp2 , server_x2 = CompressSensing_2_server( server_final_weights_part_2 )

temp
[ -0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j           0.         +0.j
 -24.77241316-17.99821168j   0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
   0.         +0.j           0.         +0.j
   0.         -0.j           0.         +0.j
   0.         +0.j           0.         +0.j
   0.         +0.j          -0.         +0.j
  -0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
  -0.         +0.j           0.         +0.j
  -0.         +0.j         -14.82286649 -4.3669172j
  -0.         +0.j          -0.         +0.j
   0.         +0.j          -0.         +0.j
  -0.         +0.j          -0.42733259+15.44683539j
  -0.         +0.j           0.         -0.j
  -0.         +0.j           0.         -0.j
   0.         +0.j          -0.         +0.j
   0.         -0.j          -0.    

temp
[ -0.        +0.j           0.        +0.j
   0.        +0.j           0.        +0.j
   0.        +0.j           0.        +0.j
   0.        +0.j           0.        +0.j
 -12.59984217-9.15432118j   0.        +0.j
   0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
   0.        -0.j           0.        +0.j
   0.        +0.j           0.        +0.j
   0.        +0.j          -0.        +0.j
  -0.        +0.j          -0.        +0.j
   0.        +0.j           0.        +0.j
  -0.        +0.j           0.        +0.j
  -0.        +0.j          -3.97021416+0.8922003j
  -0.        +0.j          -0.        +0.j
   0.        +0.j          -0.        +0.j
  -0.        +0.j          -2.07539656+3.50019299j
  -0.        +0.j           0.        -0.j
  -0.        +0.j           0.        -0.j
   0.        +0.j          -0.        +0.j
   0.        -0.j          -0.        +0.j
   0.        -0.j           0.    

In [ ]:
# calling L1 norm
server_recieved_weights1 = L1_norm1_client(server_temp1 , server_x1)
server_recieved_weights2 = L1_norm2_client(server_temp2 , server_x2)

In [ ]:
# construct , concentrate and rever_modification call to get weights

# step 1 constructing weights
recons_cons_server_temp1 = construct_1_client(server_recieved_weights1)
recons_cons_server_temp2 = construct_2_client(server_recieved_weights2)

# step 2 concentrate
server_recieved_concentrate = concentrate_client(recons_cons_server_temp1, recons_cons_server_temp2)

# step 3  reverse_modification
server_recieved_reverse_modif = reverse_modification_client(server_recieved_concentrate , sign_data_server)

Weights 1 : [0.09895963597962157, 0.9878113816072637, 0.10600477829487176, 0.6349559568590104, 0.8280616335575112, 0.4244791168825443, 0.10671528784403789]
Weights 1 : [0.20243368949842017, 0.3086139719030835, 0.10489250762452745, 0.32652068564016723, 0.24563120217886592, 0.20430287712273654, 0.3454311362091967]
Concentrated weights : [0.09895964 0.98781138 0.10600478 0.63495596 0.82806163 0.42447912
 0.10671529 0.20243369 0.30861397 0.10489251 0.32652069 0.2456312
 0.20430288 0.34543114]
reverse modified weights : [ 0.61774563  6.16631377 -0.66172423 -3.96364907  5.16909194 -2.64976844
 -0.66615951  1.26367206 -1.92649186  0.65478099  2.03827273  1.53332822
  1.27534028 -2.15631932]


In [ ]:
final_cloud_weights = copy.deepcopy(server_recieved_reverse_modif)

# CLIENT Calling :- new weights

In [ ]:
partial_model_client(data1,final_cloud_weights)

splitting dataset into train and test sets...
testing the model...
accuracy on test dataset 
 accuracy: 0.9473684210526315 
 recall : 0.8604651162790697 
 precision: 1.0


In [ ]:
partial_model_client(data2,final_cloud_weights)

splitting dataset into train and test sets...
testing the model...
accuracy on test dataset 
 accuracy: 0.9464285714285714 
 recall : 0.8536585365853658 
 precision: 1.0


In [ ]:
partial_model_client(data3,final_cloud_weights)

splitting dataset into train and test sets...
testing the model...
accuracy on test dataset 
 accuracy: 0.9727272727272728 
 recall : 0.9032258064516129 
 precision: 1.0


In [ ]:
partial_model_client(data4,final_cloud_weights)

splitting dataset into train and test sets...
testing the model...
accuracy on test dataset 
 accuracy: 0.9473684210526315 
 recall : 0.8604651162790697 
 precision: 1.0


In [ ]:
partial_model_client(data5,final_cloud_weights)

splitting dataset into train and test sets...
testing the model...
accuracy on test dataset 
 accuracy: 0.9724770642201835 
 recall : 0.9117647058823529 
 precision: 1.0
